In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
df = pd.read_csv('CombinedBettingData3.csv')
df.head()

,Unnamed: 0,HomeTeam,AwayTeam,HTWin,HTLoss,HT%,ATWin,ATLoss,AT%,Date,WinMargin,TopLine,TLCompany,BotLine,BLCompany,Average,NumComps
0,0,SAC,UTA,11.0,40.0,0.216,29.0,22.0,0.569,2009-02-06,-4,-2,Pinnacle Sports,-3,Sportsbetting,-2.875,8
1,1,POR,UTA,29.0,17.0,0.630,26.0,22.0,0.542,2009-01-31,14,9,5Dimes,9,5Dimes,9.000,8
2,2,MEM,UTA,11.0,28.0,0.282,24.0,16.0,0.600,2009-01-16,-10,-4,5Dimes,-4,5Dimes,-4.000,8
3,3,UTA,IND,23.0,15.0,0.605,13.0,25.0,0.342,2009-01-12,7,10,Bovada,9,Sportsbetting,9.250,8
4,4,HOU,UTA,20.0,11.0,0.645,18.0,14.0,0.563,2008-12-27,5,9,5Dimes,9,5Dimes,9.000,8


In [2]:
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df.columns

Index(['HomeTeam', 'AwayTeam', 'HTWin', 'HTLoss', 'HT%', 'ATWin', 'ATLoss',
       'AT%', 'Date', 'WinMargin', 'TopLine', 'TLCompany', 'BotLine',
       'BLCompany', 'Average', 'NumComps'],
      dtype='object')

In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.week
df['Weekday'] = df['Date'].dt.dayofweek
df = df.sort_values(by=['Date'])
df = df.reset_index()
df = df.drop(['index','Date'], axis=1)
df.head()

,HomeTeam,AwayTeam,HTWin,HTLoss,HT%,ATWin,ATLoss,AT%,WinMargin,TopLine,TLCompany,BotLine,BLCompany,Average,NumComps,Year,Month,Week,Weekday
0,MIN,SAC,1.0,0.0,1.0,0.0,1.0,0.0,9,5,Pinnacle Sports,4,Bookmaker,4.75,4,2006,11,44,2
1,NJN,TOR,1.0,0.0,1.0,0.0,1.0,0.0,10,7,Bookmaker,7,Bookmaker,7.00,4,2006,11,44,2
2,MEM,NYK,0.0,1.0,0.0,1.0,0.0,1.0,-1,5,Bovada,4,5Dimes,4.75,4,2006,11,44,2
3,ORL,CHI,1.0,0.0,1.0,1.0,1.0,0.5,15,-1,Bookmaker,-2,Bovada,-1.25,4,2006,11,44,2
4,CLE,WAS,1.0,0.0,1.0,0.0,1.0,0.0,3,6,Bovada,6,Bovada,6.00,4,2006,11,44,2


In [4]:
df['average'] = df['Average']
df = df.drop(['Average'], axis=1)
cols_at_end = ['WinMargin']
df = df[[c for c in df if c not in cols_at_end] + [c for c in cols_at_end if c in df]]
team = df.HomeTeam.unique()
team_dict = dict(zip(team, range(len(team))))
df1 = df.copy(deep=True)
df1 = df1.replace({'HomeTeam': team_dict})
df1 = df1.replace({'AwayTeam': team_dict})
better = df1.TLCompany.unique()
bet_dict = dict(zip(better, range(len(better))))
df1 = df1.replace({'TLCompany': bet_dict})
df1 = df1.replace({'BLCompany': bet_dict})
df1.head()

,HomeTeam,AwayTeam,HTWin,HTLoss,HT%,ATWin,ATLoss,AT%,TopLine,TLCompany,BotLine,BLCompany,NumComps,Year,Month,Week,Weekday,average,WinMargin
0,0,27,1.0,0.0,1.0,0.0,1.0,0.0,5,0,4,1,4,2006,11,44,2,4.75,9
1,1,14,1.0,0.0,1.0,0.0,1.0,0.0,7,1,7,1,4,2006,11,44,2,7.00,10
2,2,23,0.0,1.0,0.0,1.0,0.0,1.0,5,2,4,3,4,2006,11,44,2,4.75,-1
3,3,17,1.0,0.0,1.0,1.0,1.0,0.5,-1,1,-2,2,4,2006,11,44,2,-1.25,15
4,4,21,1.0,0.0,1.0,0.0,1.0,0.0,6,2,6,2,4,2006,11,44,2,6.00,3


In [5]:
df2 = df1.copy(deep=True)
df2.loc[df2['WinMargin'] < 0, 'HTLoss'] = df2['HTLoss']-1
df2.loc[df2['WinMargin'] >= 0, 'HTWin'] = df2['HTWin']-1
df2.loc[df2['WinMargin'] < 0, 'ATWin'] = df2['ATWin']-1
df2.loc[df2['WinMargin'] >= 0, 'ATLoss'] = df2['ATLoss']-1
df2['HT%'] = df2['HTWin']/(df2['HTWin'] + df2['HTLoss'])
df2['AT%'] = df2['ATWin']/(df2['ATWin'] + df2['ATLoss'])
df2.head()

,HomeTeam,AwayTeam,HTWin,HTLoss,HT%,ATWin,ATLoss,AT%,TopLine,TLCompany,BotLine,BLCompany,NumComps,Year,Month,Week,Weekday,average,WinMargin
0,0,27,0.0,0.0,NaN,0.0,0.0,NaN,5,0,4,1,4,2006,11,44,2,4.75,9
1,1,14,0.0,0.0,NaN,0.0,0.0,NaN,7,1,7,1,4,2006,11,44,2,7.00,10
2,2,23,0.0,0.0,NaN,0.0,0.0,NaN,5,2,4,3,4,2006,11,44,2,4.75,-1
3,3,17,0.0,0.0,NaN,1.0,0.0,1.0,-1,1,-2,2,4,2006,11,44,2,-1.25,15
4,4,21,0.0,0.0,NaN,0.0,0.0,NaN,6,2,6,2,4,2006,11,44,2,6.00,3


In [6]:
df2.loc[df2['WinMargin'] > df2['average'], 'Over'] = 1
df2.loc[df2['WinMargin'] <= df2['average'], 'Over'] = 0
df2 = df2.drop(['WinMargin'], axis=1)
df2 = df2.reset_index()
df2 = df2.drop(['index'], axis=1)
df2 = df2.fillna(0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
cols = df2.columns.values
cols = np.delete(cols,np.where(cols=='Over'))
df2 = df2.reset_index()
df2 = df2.drop(['index'], axis=1)
df2 = df2.fillna(0)
sc2 = StandardScaler()
for col in cols:
    df2[[col]] = sc2.fit_transform(df2[[col]])

In [7]:
train=df2[0:(round(len(df2)*.8))] 
test=df2[(round(len(df2)*.8)):]
trains = np.split(train, [18], axis=1)
train_x=trains[0]
train_y=trains[1]
tests = np.split(test, [18], axis=1)
test_x=tests[0]
test_y=tests[1]

In [8]:
def adj_r2_score(r2, n, k):
    return 1-((1-r2)*((n-1)/(n-k-1)))

In [9]:
test_y.tail()

,Over
14898,0.0
14899,0.0
14900,1.0
14901,0.0
14902,0.0


In [10]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

## Decision Trees

In [11]:
from sklearn import tree

model = tree.DecisionTreeClassifier(max_depth=10)
model.fit(train_x.values, train_y.values)
y_pred = model.predict(test_x.values)
y_pred2 = model.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

The train accuracy is 61.19778560644187
The test accuracy is 50.9560550150956


## Random Forrest

In [12]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators=100, )
model2.fit(train_x.values, train_y.values)
y_pred = model2.predict(test_x.values)
y_pred2 = model2.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


The train accuracy is 100.0
The test accuracy is 49.781952364978196


## Extra Trees

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

model3 = ExtraTreesClassifier(n_estimators=100, bootstrap=True, )
model3.fit(train_x.values, train_y.values)
y_pred = model3.predict(test_x.values)
y_pred2 = model3.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


The train accuracy is 100.0
The test accuracy is 50.72123448507212


## XGBoost

In [14]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [15]:
from xgboost import XGBClassifier

model4 = XGBClassifier(n_jobs=5,n_estimators=100,gamma=0.1,learning_rate=0.04)
model4.fit(train_x.values, train_y.values)
y_pred = model4.predict(test_x.values)
y_pred2 = model4.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The train accuracy is 57.15483979198122
The test accuracy is 50.754780275075476


## K Nearest Neighbors

In [16]:
from sklearn import neighbors

model6 = neighbors.KNeighborsClassifier()
model6.fit(train_x.values, train_y.values)
y_pred = model6.predict(test_x.values)
y_pred2 = model6.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


The train accuracy is 69.37594363361852
The test accuracy is 50.285139215028515


## TPOT

In [18]:
from tpot import TPOTClassifier
from sklearn.metrics.scorer import make_scorer

#my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)

tpot = TPOTClassifier(population_size=10, max_time_mins=1, verbosity=1, config_dict='TPOT light')
tpot.fit(train_x.values, train_y.values.ravel())


1.0100565 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
Best pipeline: DecisionTreeClassifier(Normalizer(input_matrix, norm=max), criterion=gini, max_depth=3, min_samples_leaf=7, min_samples_split=11)


TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=1000000,
        max_eval_time_mins=5, max_time_mins=1, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=10,
        random_state=None, scoring=None, subsample=1.0,
        template='RandomTree', use_dask=False, verbosity=1,
        warm_start=False)

In [19]:
y_pred = tpot.predict(test_x.values)
y_pred2 = tpot.predict(train_x.values)
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

The train accuracy is 52.13051501425935
The test accuracy is 50.72123448507212


## Neural Nets

In [20]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
from keras import regularizers

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [21]:
K.clear_session()
model = Sequential()
model.add(Dense(64, input_dim=18, activation='tanh', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(0.13)))
Dropout(.2)
model.add(Dense(34, activation='tanh',kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(0.03)))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics =['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0, min_delta=.000001,restore_best_weights=True)
        
out = model.fit(train_x.values, train_y.Over.values.ravel(), validation_data=[test_x.values, test_y['Over'].values.ravel()], epochs=100,verbose=1,shuffle=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 11922 samples, validate on 2981 samples
Epoch 1/100
11922/11922 [==============================] - 2s 193us/step - loss: 1.0794 - acc: 0.4876 - val_loss: 0.8068 - val_acc: 0.4951
Epoch 2/100
11922/11922 [==============================] - 1s 74us/step - loss: 0.8044 - acc: 0.5026 - val_loss: 0.7863 - val_acc: 0.4951
Epoch 3/100
11922/11922 [==============================] - 1s 71us/step - loss: 0.7867 - acc: 0.5016 - val_loss: 0.7721 - val_acc: 0.4951
Epoch 4/100
11922/11922 [==============================] - 1s 74us/step - loss: 0.7709 - acc: 0.5009 - val_loss: 0.7573 - val_acc: 0.4951
Epoch 5/100
11922/11922 [==============================] - 1s 72us/step - loss: 0.7556 - acc: 0.4987 - val_loss: 0.7428 - val_acc: 0.4951
Epoch 6/100
11922/11922 [==============================] - 1s 72us/step - loss: 0.7425 - acc: 0.5016 - val_loss: 0.7271 - val_acc: 0.4951
Ep

Epoch 56/100
11922/11922 [==============================] - 1s 72us/step - loss: 0.6985 - acc: 0.5041 - val_loss: 0.7076 - val_acc: 0.4951
Epoch 57/100
11922/11922 [==============================] - 1s 70us/step - loss: 0.6988 - acc: 0.5043 - val_loss: 0.7099 - val_acc: 0.4951
Epoch 58/100
11922/11922 [==============================] - 1s 71us/step - loss: 0.6984 - acc: 0.5034 - val_loss: 0.7089 - val_acc: 0.4951
Epoch 59/100
11922/11922 [==============================] - 1s 73us/step - loss: 0.6982 - acc: 0.5031 - val_loss: 0.7103 - val_acc: 0.4951
Epoch 60/100
11922/11922 [==============================] - 1s 72us/step - loss: 0.6981 - acc: 0.5043 - val_loss: 0.7026 - val_acc: 0.4951
Epoch 61/100
11922/11922 [==============================] - 1s 70us/step - loss: 0.6981 - acc: 0.5056 - val_loss: 0.7059 - val_acc: 0.4951
Epoch 62/100
11922/11922 [==============================] - 1s 70us/step - loss: 0.6981 - acc: 0.5041 - val_loss: 0.7043 - val_acc: 0.4951
Epoch 63/100
11922/11922 [=

In [22]:
y_pred = np.rint(model.predict(test_x.values))
y_pred2 = np.rint(model.predict(train_x.values))
trainaccuracy =  accuracy_score(train_y.values, y_pred2) * 100
testaccuracy =  accuracy_score(test_y.values, y_pred) * 100
print('The train accuracy is ' + str(trainaccuracy))
print('The test accuracy is ' + str(testaccuracy))

The train accuracy is 48.465022647206844
The test accuracy is 49.51358604495136
